# LeetCode 99: Recover Binary Search Tree（Python版）

## 1. 問題分析結果

### 競技プログラミング視点
- **制約分析**: n ≤ 1000 → O(n²)でも余裕、O(n)が最適
- **最速手法**: 再帰的中順走査（CPythonで最適化済み）
- **メモリ最小化**: Morris TraversalでO(1)空間可能
- **CPython最適化**: 関数呼び出しは比較的軽量、nonlocalで外部変数アクセス

### 業務開発視点
- **型安全設計**: `Optional[TreeNode]`による明示的なnull安全性
- **エラーハンドリング**: 不正なツリー構造への防御
- **可読性**: 明確な変数名、docstring完備

### Python特有分析
- **再帰制限**: `sys.setrecursionlimit()` - n≤1000では不要
- **nonlocal**: クロージャ内で外部変数を変更可能
- **タプルアンパック**: `a, b = b, a` でスワップ（最速）

---

## 2. アルゴリズム比較表

| アプローチ | 時間計算量 | 空間計算量 | Python実装コスト | 可読性 | CPython最適化 | 備考 |
|-----------|-----------|-----------|-----------------|--------|--------------|------|
| 方法A: 配列保存 | O(n) | O(n) | 低 | ★★★ | 適 | リスト内包表記活用可 |
| 方法B: 再帰走査 | O(n) | O(h) | 低 | ★★★ | 適 | nonlocalで状態管理 |
| 方法C: Morris走査 | O(n) | O(1) | 中 | ★★☆ | 適 | ポインタ操作のみ |

---

## 3. 採用アルゴリズムと根拠

- **選択**: 方法B（再帰版）+ 方法C（Morris版）
- **理由**:
  - 再帰版: Pythonで最も自然、可読性最高
  - Morris版: Follow-up要件（O(1)空間）対応
- **Python最適化戦略**:
  - `nonlocal`で変数共有（クラス属性より高速）
  - タプルスワップ `a, b = b, a`
  - 早期リターンで無駄な処理回避

---

## 4. 実装コード

```python
"""
LeetCode 99: Recover Binary Search Tree
Python (CPython 3.11.10) 実装
"""
from __future__ import annotations
from typing import Optional


# Definition for a binary tree node.
class TreeNode:
    """Binary Tree Node Definition"""
    __slots__ = ('val', 'left', 'right')
    
    def __init__(
        self,
        val: int = 0,
        left: Optional[TreeNode] = None,
        right: Optional[TreeNode] = None
    ) -> None:
        self.val = val
        self.left = left
        self.right = right


class Solution:
    """
    Recover Binary Search Tree
    
    2つのノードが誤って入れ替わったBSTを修復する
    """
    
    # ============ 競技プログラミング向け（最速・推奨） ============
    def recoverTree(self, root: Optional[TreeNode]) -> None:
        """
        競技プログラミング向け最適化実装
        再帰的中順走査で違反ノードを検出
        
        Time Complexity: O(n)
        Space Complexity: O(h) where h is tree height
        """
        first: Optional[TreeNode] = None
        second: Optional[TreeNode] = None
        prev: Optional[TreeNode] = None
        
        def dfs(node: Optional[TreeNode]) -> None:
            nonlocal first, second, prev
            if not node:
                return
            
            dfs(node.left)
            
            if prev and prev.val > node.val:
                if not first:
                    first = prev
                second = node
            prev = node
            
            dfs(node.right)
        
        dfs(root)
        
        # Swap values (first and second are guaranteed to exist)
        if first and second:
            first.val, second.val = second.val, first.val
    
    # ============ Morris Traversal（O(1)空間） ============
    def recoverTreeMorris(self, root: Optional[TreeNode]) -> None:
        """
        Morris In-order Traversal実装
        定数空間でBSTを修復
        
        Time Complexity: O(n)
        Space Complexity: O(1)
        """
        first: Optional[TreeNode] = None
        second: Optional[TreeNode] = None
        prev: Optional[TreeNode] = None
        curr: Optional[TreeNode] = root
        
        while curr:
            if curr.left:
                # Find predecessor
                pred: TreeNode = curr.left
                while pred.right and pred.right is not curr:
                    pred = pred.right
                
                if not pred.right:
                    # Create thread
                    pred.right = curr
                    curr = curr.left
                    continue
                
                # Remove thread
                pred.right = None
            
            # Process current node
            if prev and prev.val > curr.val:
                if not first:
                    first = prev
                second = curr
            prev = curr
            curr = curr.right
        
        # Swap values
        if first and second:
            first.val, second.val = second.val, first.val
    
    # ============ 業務開発向け（型安全・エラーハンドリング） ============
    def recoverTreeProduction(self, root: Optional[TreeNode]) -> None:
        """
        業務開発向け実装
        型安全性・エラーハンドリング・可読性重視
        
        Args:
            root: BSTのルートノード
            
        Raises:
            ValueError: ルートがNoneの場合
            RuntimeError: 入れ替わったノードが見つからない場合
        """
        # 入力検証
        if root is None:
            raise ValueError("Root cannot be None")
        
        # 違反ノードを格納
        swapped_nodes: list[TreeNode] = []
        prev_node: list[Optional[TreeNode]] = [None]  # ミュータブルコンテナ
        
        def inorder_traverse(node: Optional[TreeNode]) -> None:
            """中順走査で違反ノードを検出"""
            if node is None:
                return
            
            # 左部分木を走査
            inorder_traverse(node.left)
            
            # 違反チェック
            prev = prev_node[0]
            if prev is not None and prev.val > node.val:
                if len(swapped_nodes) == 0:
                    swapped_nodes.append(prev)
                # 常に現在のノードを第2候補として更新
                if len(swapped_nodes) == 1:
                    swapped_nodes.append(node)
                else:
                    swapped_nodes[1] = node
            
            prev_node[0] = node
            
            # 右部分木を走査
            inorder_traverse(node.right)
        
        # 走査実行
        inorder_traverse(root)
        
        # 結果検証とスワップ
        if len(swapped_nodes) != 2:
            raise RuntimeError(
                f"Expected 2 swapped nodes, found {len(swapped_nodes)}"
            )
        
        first_node, second_node = swapped_nodes
        first_node.val, second_node.val = second_node.val, first_node.val
    
    # ============ イテレーティブスタック版 ============
    def recoverTreeIterative(self, root: Optional[TreeNode]) -> None:
        """
        明示的スタックを使用したイテレーティブ実装
        再帰深度制限を回避
        
        Time Complexity: O(n)
        Space Complexity: O(h)
        """
        stack: list[TreeNode] = []
        first: Optional[TreeNode] = None
        second: Optional[TreeNode] = None
        prev: Optional[TreeNode] = None
        curr: Optional[TreeNode] = root
        
        while curr or stack:
            # 左端まで進む
            while curr:
                stack.append(curr)
                curr = curr.left
            
            # ノードを処理
            curr = stack.pop()
            
            if prev and prev.val > curr.val:
                if not first:
                    first = prev
                second = curr
            prev = curr
            
            # 右に移動
            curr = curr.right
        
        if first and second:
            first.val, second.val = second.val, first.val
```

---

## 5. LeetCode提出用コード（順番に試してください）

### 🥇 最速版（推奨）

```python
class Solution:
    def recoverTree(self, root: Optional[TreeNode]) -> None:
        first = second = prev = None
        
        def dfs(node: Optional[TreeNode]) -> None:
            nonlocal first, second, prev
            if not node:
                return
            dfs(node.left)
            if prev and prev.val > node.val:
                if not first:
                    first = prev
                second = node
            prev = node
            dfs(node.right)
        
        dfs(root)
        first.val, second.val = second.val, first.val

Analyze Complexity
Runtime 3 ms
Beats 59.86%
Memory 20.64 MB
Beats 19.96%

```

### 🥈 イテレーティブ版

```python
class Solution:
    def recoverTree(self, root: Optional[TreeNode]) -> None:
        stack = []
        first = second = prev = None
        curr = root
        
        while curr or stack:
            while curr:
                stack.append(curr)
                curr = curr.left
            curr = stack.pop()
            if prev and prev.val > curr.val:
                if not first:
                    first = prev
                second = curr
            prev = curr
            curr = curr.right
        
        first.val, second.val = second.val, first.val

Analyze Complexity
Runtime 0 ms
Beats 100.00%
Memory 19.56 MB
Beats 19.96%

```

### 🥉 Morris版（O(1)空間）

```python
class Solution:
    def recoverTree(self, root: Optional[TreeNode]) -> None:
        first = second = prev = None
        curr = root
        
        while curr:
            if curr.left:
                pred = curr.left
                while pred.right and pred.right is not curr:
                    pred = pred.right
                if not pred.right:
                    pred.right = curr
                    curr = curr.left
                    continue
                pred.right = None
            
            if prev and prev.val > curr.val:
                if not first:
                    first = prev
                second = curr
            prev = curr
            curr = curr.right
        
        first.val, second.val = second.val, first.val

Analyze Complexity
Runtime 1 ms
Beats 69.65%
Memory 19.58 MB
Beats 19.96%

```

---

## 6. Python特有の最適化ポイント

| 最適化 | 実装 | 効果 |
|--------|------|------|
| タプルスワップ | `a, b = b, a` | 一時変数不要、C実装で高速 |
| nonlocal | クロージャ内で外部変数変更 | クラス属性より高速 |
| `is not` vs `!=` | `pred.right is not curr` | アイデンティティ比較で高速 |
| `__slots__` | TreeNodeクラスで使用 | メモリ削減（LeetCodeでは不要） |
| 早期continue | Morris版で使用 | ネスト削減、可読性向上 |

---

## 7. 型安全性（Pylance対応）

```python
# ✅ Pylance対応済み
first: Optional[TreeNode] = None  # 明示的な型注釈
second: Optional[TreeNode] = None
prev: Optional[TreeNode] = None

# ✅ 最終行でのスワップ（型チェッカーへの対応）
if first and second:  # None チェック後にアクセス
    first.val, second.val = second.val, first.val

# または（問題で2ノードの入れ替えが保証されている場合）
first.val, second.val = second.val, first.val  # type: ignore
```

---

## 期待される結果

| 指標 | 期待値 |
|------|--------|
| Runtime | 30-50ms (85-95%) |
| Memory | 16-18MB (70-90%) |

Pythonでは再帰版が最もシンプルで高速です。まず最速版を試してください。

はい、メモリの改善余地があります。Pythonの特性を考慮した最適化を行います。

## メモリ使用量が高い原因分析

| 原因 | 影響 | 改善策 |
|------|------|--------|
| 型ヒント | 実行時にオブジェクト生成 | 提出時は削除 |
| `nonlocal`クロージャ | クロージャオブジェクトのオーバーヘッド | インスタンス変数使用 |
| 関数定義 | ネストした関数オブジェクト | クラスメソッドに移行 |
| 複数変数宣言 | 各変数がオブジェクト | 最小化 |

## メモリ最適化版コード## 提出用コード（順番に試してください）

### 🥇 まず試す: インスタンス変数版（クロージャ回避）

```python
class Solution:
    def recoverTree(self, root):
        self.first = self.second = self.prev = None
        self._dfs(root)
        self.first.val, self.second.val = self.second.val, self.first.val

    def _dfs(self, node):
        if not node:
            return
        self._dfs(node.left)
        if self.prev and self.prev.val > node.val:
            if not self.first:
                self.first = self.prev
            self.second = node
        self.prev = node
        self._dfs(node.right)

Analyze Complexity
Runtime 3 ms
Beats 59.86%
Memory 19.57 MB
Beats 19.96%

```

### 🥈 次に試す: Morris版（理論上O(1)空間）

```python
class Solution:
    def recoverTree(self, root):
        f = s = p = None
        c = root
        while c:
            if c.left:
                pred = c.left
                while pred.right and pred.right is not c:
                    pred = pred.right
                if not pred.right:
                    pred.right = c
                    c = c.left
                    continue
                pred.right = None
            if p and p.val > c.val:
                if not f:
                    f = p
                s = c
            p = c
            c = c.right
        f.val, s.val = s.val, f.val

Analyze Complexity
Runtime 1 ms
Beats 69.65%
Memory 20.56 MB
Beats 19.96%

```

### 🥉 イテレーティブ版（スタック使用）

```python
class Solution:
    def recoverTree(self, root):
        stack = []
        f = s = p = None
        c = root
        while c or stack:
            while c:
                stack.append(c)
                c = c.left
            c = stack.pop()
            if p and p.val > c.val:
                if not f:
                    f = p
                s = c
            p = c
            c = c.right
        f.val, s.val = s.val, f.val

Analyze Complexity
Runtime 1 ms
Beats 69.65%
Memory 19.58 MB
Beats 19.96%

```

---

## 最適化ポイント詳細

| 変更点 | Before | After | メモリ効果 |
|--------|--------|-------|-----------|
| 型ヒント | `Optional[TreeNode]` | 削除 | 型オブジェクト生成回避 |
| 状態管理 | `nonlocal` クロージャ | `self.` インスタンス変数 | クロージャオーバーヘッド削減 |
| 変数名 | `first, second, prev` | `f, s, p` | 文字列インターン削減（微小） |
| 関数定義 | ネストした関数 | クラスメソッド | 関数オブジェクト削減 |

---

## なぜメモリが高いのか（Python特性）

```
Pythonのメモリ構造:
┌─────────────────────────────────────┐
│ PyObject Header (16 bytes)         │ ← 全オブジェクトに必要
├─────────────────────────────────────┤
│ Type pointer                       │
│ Reference count                    │
├─────────────────────────────────────┤
│ Actual data                        │
└─────────────────────────────────────┘

クロージャの場合:
┌─────────────────────────────────────┐
│ Function object                    │
├─────────────────────────────────────┤
│ __closure__ (cell objects)         │ ← nonlocal変数分の追加メモリ
│   - cell for 'first'               │
│   - cell for 'second'              │
│   - cell for 'prev'                │
└─────────────────────────────────────┘
```

---

## 期待される改善

| 指標 | 現在 | 期待値 |
|------|------|--------|
| Memory (再帰版) | 20.64MB (19.96%) | 17-18MB (50-70%) |
| Memory (Morris版) | 19.58MB (19.96%) | 16-17MB (60-80%) |

**補足**: Pythonではインタープリタ自体のオーバーヘッドがあるため、メモリ改善には限界があります。LeetCodeの他の高ランク解答も似たようなコードを使っているため、19-20MB前後が実質的な下限かもしれません。

複数回提出を試してみてください。LeetCodeのメモリ計測にはブレがあります。